Copyright (c) 2020 Martin Holle. Alle Rechte vorbehalten. Lizensiert unter der MIT-Lizenz.


# Covid-19 Statistics Aachen: Upload

Die im vorherigen Schritt produzierten Grafik-Dateien und `yml`-Dateien mit den Meta-Daten werden in diesem Schritt per FTP in das Zielverzeichnis auf dem Web-Server hochgeladen.

## Benötigte Imports und Vorbereitungen

In [1]:
import logging
import configparser
import os
import ftplib
from ftplib import FTP

# Konfiguration einlesen
config = configparser.ConfigParser()
config.read('config.ini')
config.read('upload.ini')

# Konfiguration des Loggings
# - Die Logging-Ausgaben werden in der lokalen Datei covid-19-datenabfrage.log geschrieben
# - Für die Ausgabe wird eine bestimmte Formatierung konfiguriert
fhandler = logging.FileHandler(filename=config['Logging']['LogFileName'], mode='a')

# TODO: Formatierung finalisieren (Tausendstel-Sekunden, Tag des Monats, 1. Zeichen des Levels)
formatter = logging.Formatter('%(asctime)s %(levelname)-1.1s %(name)-20.20s - %(message)s')
fhandler.setFormatter(formatter)

# Instanzierung und Konfigurierung des Loggers
log = logging.getLogger("upload")
log.addHandler(fhandler)
log.setLevel(logging.INFO)

## Upload

In [22]:
if (config.has_section('Upload')):
    output_folder=config['Visualisierung']['OutputFolder']

    host = config['Upload']['ftp.hostaddress']
    user = config['Upload']['ftp.username']
    pwd = config['Upload']['ftp.password']
    target_folder = config['Upload']['ftp.targetdir']    

    files = os.listdir(output_folder)
    
    log.info(f"Start uploading %d files from local folder: %s", len(files), output_folder)

    uploaded_bytes = 0
    def count_bytes(chunk):
        global uploaded_bytes
        uploaded_bytes += len(chunk)
        print(".", end='')
        
    try:
        with FTP(host) as ftp:
            ftp.login(user=user, passwd=pwd)
            log.info(f"Logged in to FTP host %s", host)
            ftp.cwd(target_folder)
            log.info(f"Target folder: %s", target_folder)
            print("Uploading started")
            for filename in files:
                log.info(f"Start uploading file: %s", filename)
                with open(os.path.join(output_folder, filename), "rb") as file:
                    print(filename + ' ', end='')
                    ftp.storbinary("STOR {0}".format(filename), file, callback=count_bytes, blocksize=16384)
                    print(" {0} bytes".format(str(uploaded_bytes)))
            print("Uploading stopped")
            
        log.info(f"Uploaded %d bytes to FTP host", uploaded_bytes)
            
    except OSError as ose:
        log.error(f"File error '%s' (%s) during FTP upload", str(ose.errno), ose.strerror)
    
    except ftplib.all_errors as fe:
        log.error(f"FTP error '%s' during FTP upload", str(fe).split(None, 1)[0])

    log.info('Uploading stopped')
else:
    log.warning('No upload configuration file found.')

Uploading started
c19-diagram-1.jpg ............................... 498204 bytes
c19-diagram-2.jpg ........................................................ 1410041 bytes
c19-diagram-3.jpg ............................................ 2119126 bytes
c19-diagram-4.jpg ................................... 2690217 bytes
Uploading stopped
